In [1]:
import job_scraper
# from job_scraper import find_jobs_from, load_indeed_jobs_div, load_hinge_jobs_div
import urllib
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import os

In [2]:
print(job_scraper)

<module 'job_scraper' from 'C:\\Users\\lauwx1\\Desktop\\job-scraper\\job_scraper.py'>


In [3]:
desired_characs = ['titles', 'companies', 'links', 'date_listed']

## Extracting jobs from Indeed.co.uk

In [ ]:
find_jobs_from('Indeed', 'data analyst', 'california', desired_characs)

## Extracting jobs from Hinge Health

In [ ]:
# load_indeed_jobs_div('data analyst','california')

In [42]:
def load_hinge_jobs_div(commitment='Full-time', team='Engineering'):
    getVars = {'commitment': commitment, 'team': team}
    url = ('https://jobs.lever.co/hingehealth?' + urllib.parse.urlencode(getVars))
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    # job_soup = soup.find(id="resultsCol")
    # job_soup = soup.find(data-qa="posting-name")
#     job_soup = soup.find_all('h5') ##this returns all the job titles/names, but try to get the LINKS posted as well
#     job_soup = soup.find_all('a', class_="posting-title")##trying to get the link for job
    job_soup = soup.find(class_="postings-group")
    return job_soup

In [14]:
# getVars = {'commitment': 'Full-time', 'team': 'Engineering'}
# urllib.parse.urlencode(getVars)

'commitment=Full-time&team=Engineering'

In [43]:
# find_jobs_from('Hinge', 'data analyst', 'california', desired_characs)
test_soup = load_hinge_jobs_div(commitment='Full-time', team='Engineering') ## find_all funct returns a <class 'bs4.element.ResultSet'>
# page = requests.get('https://jobs.lever.co/hingehealth?commitment=Full-time&team=Engineering')
# soup = BeautifulSoup(page.content, "html.parser")
# soup ##somehow when i run the stuff inside the function it works, but the call to the function won't work. its calling None??

## basically try to find TITLE have it from h5 tag, and all the DATE_LISTED associated with that job
##title is under h5 tag data-qa = posting-name
## link is under a class = posting-title href.. now to extract this.

In [66]:
##extract info from soup
def extract_job_information_indeed(job_soup, desired_characs):
    job_elems = job_soup.find_all('div', class_='jobsearch-SerpJobCard')

    cols = []
    extracted_info = []

    if 'titles' in desired_characs:
        titles = []
        cols.append('titles')
        for job_elem in job_elems:
            titles.append(extract_job_title_indeed(job_elem))
        extracted_info.append(titles)

#     if 'companies' in desired_characs:
#         companies = []
#         cols.append('companies')
#         for job_elem in job_elems:
#             companies.append(extract_company_indeed(job_elem))
#         extracted_info.append(companies)

    if 'links' in desired_characs:
        links = []
        cols.append('links')
        for job_elem in job_elems:
            links.append(extract_link_indeed(job_elem))
        extracted_info.append(links)

#     if 'date_listed' in desired_characs:
#         dates = []
#         cols.append('date_listed')
#         for job_elem in job_elems:
#             dates.append(extract_date_indeed(job_elem))
#         extracted_info.append(dates)

    jobs_list = {}

    for j in range(len(cols)):
        jobs_list[cols[j]] = extracted_info[j]

    num_listings = len(extracted_info[0])

    return jobs_list, num_listings

def extract_job_title_hinge(job_elem):
    title_elem = job_elem.find('h5')
    title = title_elem.text.strip()
    return title

def extract_link_hinge(job_elem):
    link = job_elem.find('a')['href']
#     link = 'www.Indeed.co.uk/' + link
    return link


In [84]:
job_elem

<a class="posting-title" href="https://jobs.lever.co/hingehealth/ac1ae9df-8d57-415b-98dd-c27668ed91f2"><h5 data-qa="posting-name">Data Engineer</h5><div class="posting-categories"><span class="sort-by-location posting-category small-category-label" href="#">San Francisco, Portland, Denver, Austin, Los Angeles, Seattle, New York</span><span class="sort-by-team posting-category small-category-label" href="#">Engineering</span></div></a>

In [87]:
job_elem.find_all('a', class_='posting-title')

[]

In [96]:
## test extract title function
type(test_soup)
job_elems = test_soup.find_all('a', class_='posting-title')
type(job_elems)
titles = []
extracted_info = []
for job_elem in job_elems:
    titles.append(extract_job_title_hinge(job_elem))
extracted_info.append(titles)
# extract_job_title_hinge(job_elems)


##test extract link function
links = []
# for job_elem in job_elems:
#     links.append(extract_link_hinge(job_elem))
# extracted_info.append(links)

In [97]:
##compare results to the indeed query/function
extracted_info
# job_elems

[['Data Engineer',
  'Director of Application Engineering, Core App',
  'Engineering Manager (Platform)',
  'Senior Software Engineer (Backend)',
  'Senior Software Engineer (Data Focus)',
  'Senior Software Engineer (Front End)',
  'Senior Software Engineer (Full-Stack)',
  'Senior Software Engineer (Internal Tooling)',
  'Site Reliability Engineer',
  'Technical Lead']]

In [26]:
def load_indeed_jobs_div(job_title, location):
    getVars = {'q': job_title, 'l': location, 'fromage': 'last', 'sort': 'date'}
    url = ('https://www.indeed.com/jobs?' + urllib.parse.urlencode(getVars))
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    job_soup = soup.find(id="resultsCol")
    return job_soup

In [47]:
job_title="data scientist"
location = "california"
load_indeed_jobs_div(job_title,location) ## returns a <class 'bs4.element.Tag'> from "find" function
type(load_indeed_jobs_div(job_title,location))

bs4.element.Tag

## Extracting jobs from CWjobs.co.uk (using Selenium)

In [ ]:
find_jobs_from('CWjobs', 'data scientist', 'london', desired_characs)